<a href="https://colab.research.google.com/github/fred-lafleche/Exchange/blob/master/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
#imports
from sklearn import preprocessing
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plot
from sklearn.utils import resample
from scipy.sparse import coo_matrix
from sklearn.utils import resample
from sklearn import svm
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score

In [88]:
heartInput_file = "/content/drive/MyDrive/Colab Notebooks/heart.csv"
marketingInput_file = "/content/drive/MyDrive/Colab Notebooks/marketing_campaign.csv"
#10 numerical and 8 categorical attributes, where revenue is class label
#12330 rows and 18 columns
heartDataPd = pd.read_csv(heartInput_file)
marketDataPd = pd.read_csv(marketingInput_file)
heartDataPd.head()
marketDataPd.head()
#print(data["VisitorType"][1000]

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0


Heart Data

In [ ]:
heartDataArray = heartDataPd.values
print(heartDataArray.size)
heartData = np.delete(heartDataArray, 13, 1)
print(heartData.size)
heartLabel = heartDataArray[:,[13]]
print(heartLabel.size)

4242
3939
303


Marketing Campaign

In [119]:
marketDataArray = marketDataPd.values
print(marketDataArray.size)
marketData = np.delete(marketDataArray, 6, 1)

#to do: one hot encoding on row 2, 3, 7
marketData = np.delete(marketData, 6, 1)
marketData = np.delete(marketData, 4, 1)
marketData = np.delete(marketData, 3, 1)
marketData = np.delete(marketData, 2, 1)


print(marketData[0])
print(marketData.size)
marketLabel = marketDataArray[:,[6]]
print(marketLabel.size)

64960
[5524 1957 0 58 635 88 546 172 88 88 3 8 10 4 7 0 0 0 0 0 0 3 11 1]
53760
2240


# Single Decision Tree

In [120]:
#Heart data
SDT = tree.DecisionTreeClassifier()
SDTscores = cross_val_score(SDT,heartData,heartLabel,cv=10)
print("Heart Data Accuracy: %0.2f (+/- %0.2f)" % (SDTscores.mean(), SDTscores.std() * 2))

#Marketing data
SDT2 = tree.DecisionTreeClassifier()
SDTscores2 = cross_val_score(SDT2,marketData,marketLabel,cv=10)
print("Marketing Data Accuracy: %0.2f (+/- %0.2f)" % (SDTscores2.mean(), SDTscores2.std() * 2))


Heart Data Accuracy: 0.76 (+/- 0.14)
Marketing Data Accuracy: nan (+/- nan)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unknown label type: 'unknown'

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unknown label type: 'unknown'

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unknown label type: 'unknown'

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these 

# Random Forest

In [101]:
#Heart data
RF = RandomForestClassifier()
RFscores = cross_val_score(RF,heartData,heartLabel,cv=10)
print("Heart Data Accuracy: %0.2f (+/- %0.2f)" % (RFscores.mean(), RFscores.std() * 2))

#Marketing data
RF2 = RandomForestClassifier()
RFscores2 = cross_val_score(RF2,marketData,marketLabel,cv=10)
print("Marketing Data Accuracy: %0.2f (+/- %0.2f)" % (RFscores2.mean(), RFscores2.std() * 2))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

Heart Data Accuracy: 0.84 (+/- 0.12)
Marketing Data Accuracy: nan (+/- nan)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'Graduation'

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'Graduation'

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'Graduation'

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit fai

# Support Vector Machine

In [102]:
#Heart data
s_v_m = svm.SVC()
SVMscores = cross_val_score(s_v_m,heartData,heartLabel,cv=10)
print("Heart Data Accuracy: %0.2f (+/- %0.2f)" % (SVMscores.mean(), SVMscores.std() * 2))

#Marketing data
s_v_m2 = svm.SVC()
SVMscores2 = cross_val_score(s_v_m2,marketData,marketLabel,cv=10)
print("Marketing Data Accuracy: %0.2f (+/- %0.2f)" % (SVMscores2.mean(), SVMscores2.std() * 2))

Heart Data Accuracy: 0.66 (+/- 0.16)
Marketing Data Accuracy: nan (+/- nan)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

# K-Nearest Neighbor

In [99]:
#Heart data
knn = KNeighborsClassifier()
KNNscores = cross_val_score(knn,heartData,heartLabel,cv=10)
print("Heart Data Accuracy: %0.2f (+/- %0.2f)" % (KNNscores.mean(), KNNscores.std() * 2))

#Marketing data
knn2 = KNeighborsClassifier()
KNNscores2 = cross_val_score(knn2,marketData,marketLabel,cv=10)
print("Heart Data Accuracy: %0.2f (+/- %0.2f)" % (KNNscores2.mean(), KNNscores2.std() * 2))

Heart Data Accuracy: 0.65 (+/- 0.16)
Heart Data Accuracy: 0.65 (+/- 0.16)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when

# Multi-layer Perceptron

In [ ]:
print("Hello")

Hello


# Gradient Boosting

In [ ]:
print("Hello")

Hello
